In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv("student-mat.csv", delimiter=";")

# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=["object"]).columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Split the dataset into features and target
X = data.drop(columns=["G1", "G2", "G3"])
y = data[["G3"]]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_input_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_hidden_output = np.zeros((1, self.output_size))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def forward(self, X):
        # Forward pass from input to hidden layer
        self.hidden_output = self.relu(np.dot(X, self.weights_input_hidden) + self.bias_input_hidden)
        
        # Forward pass from hidden to output layer
        self.output = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_hidden_output
        
        return self.output
    
    def backward(self, X, y, output, learning_rate):
        # Compute gradients
        output_error = output - y
        hidden_error = np.dot(output_error, self.weights_hidden_output.T)
        
        output_delta = output_error
        hidden_delta = hidden_error * (self.hidden_output > 0)
        
        # Update weights and biases
        self.weights_hidden_output -= learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_hidden_output -= learning_rate * np.sum(output_delta, axis=0)
        self.weights_input_hidden -= learning_rate * np.dot(X.T, hidden_delta)
        self.bias_input_hidden -= learning_rate * np.sum(hidden_delta, axis=0)
        
    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            # Forward propagation
            output = self.forward(X)
            
            # Backpropagation
            self.backward(X, y, output, learning_rate)
            
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        return self.forward(X)


# Convert data to numpy arrays
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

# Define parameters
input_size = X_train_np.shape[1]
output_size = 1

# Model with 1 hidden layer and 32 nodes
model_32 = NeuralNetwork(input_size, 32, output_size)
model_32.train(X_train_np, y_train_np, learning_rate=0.001, epochs=1000)

# Model with 1 hidden layer and 64 nodes
model_64 = NeuralNetwork(input_size, 64, output_size)
model_64.train(X_train_np, y_train_np, learning_rate=0.001, epochs=1000)

Epoch 0, Loss: 1410.8116414661977
Epoch 100, Loss: 1.9422279586698806e+60
Epoch 200, Loss: 1.9930607915942976e+27
Epoch 300, Loss: 21.004518548989463
Epoch 400, Loss: 21.004516503765423
Epoch 500, Loss: 21.004516503765423
Epoch 600, Loss: 21.004516503765423
Epoch 700, Loss: 21.004516503765423
Epoch 800, Loss: 21.004516503765423
Epoch 900, Loss: 21.004516503765423
Epoch 0, Loss: 8512.950437987276
Epoch 100, Loss: 21.004516503765423
Epoch 200, Loss: 21.004516503765423
Epoch 300, Loss: 21.004516503765423
Epoch 400, Loss: 21.004516503765423
Epoch 500, Loss: 21.004516503765423
Epoch 600, Loss: 21.004516503765423
Epoch 700, Loss: 21.004516503765423
Epoch 800, Loss: 21.004516503765423
Epoch 900, Loss: 21.004516503765423


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PyTorchModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)

# Define parameters
input_size = X_train_tensor.shape[1]
output_size = 1
hidden_size = 64

# Initialize the model
pytorch_model = PyTorchModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(pytorch_model.parameters(), lr=0.001)

# Training the model
for epoch in range(1000):
    optimizer.zero_grad()
    output = pytorch_model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Prediction
with torch.no_grad():
    y_pred_tensor = pytorch_model(X_test_tensor)
    y_pred_np = y_pred_tensor.numpy()

# Convert predictions to binary classes
y_pred_binary = np.round(y_pred_np)

# Calculate accuracy
accuracy = accuracy_score(y_test_np, y_pred_binary)
print(f"Test Set Accuracy (PyTorch): {accuracy}")

Epoch 0, Loss: 133.56507873535156
Epoch 100, Loss: 19.843021392822266
Epoch 200, Loss: 18.193872451782227
Epoch 300, Loss: 16.884693145751953
Epoch 400, Loss: 15.927068710327148
Epoch 500, Loss: 15.255062103271484
Epoch 600, Loss: 14.921277046203613
Epoch 700, Loss: 15.438356399536133
Epoch 800, Loss: 15.063600540161133
Epoch 900, Loss: 14.545741081237793
Test Set Accuracy (PyTorch): 0.11392405063291139


In [7]:
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]

results = []

for lr in learning_rates:
    # Train the model with different learning rates
    model = NeuralNetwork(input_size, 64, output_size)
    model.train(X_train_np, y_train_np, learning_rate=lr, epochs=1000)
    
    # Test the model
    predictions = model.predict(X_test_np)
    accuracy = accuracy_score(y_test_np, np.round(predictions))
    results.append((lr, accuracy))

Epoch 0, Loss: 27633.585722789292
Epoch 100, Loss: inf


C:\Users\ishaa\AppData\Local\Temp\ipykernel_10876\3993830879.py:53: RuntimeWarning: overflow encountered in square
  loss = np.mean(np.square(y - output))
C:\Users\ishaa\AppData\Local\Temp\ipykernel_10876\3993830879.py:36: RuntimeWarning: invalid value encountered in multiply
  hidden_delta = hidden_error * (self.hidden_output > 0)


Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan


c:\Users\ishaa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\_array_api.py:290: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.

In [ ]:
from sklearn.metrics import classification_report

# Assuming you have trained and tested your custom model and PyTorch model
# Let's assume you have the predictions and ground truth labels for both models

# Classification report for your custom model
print("Classification Report for Your Custom Model:")
print(classification_report(y_true_custom, y_pred_custom))

# Classification report for the PyTorch model
print("Classification Report for PyTorch Model:")
print(classification_report(y_true_pytorch, y_pred_pytorch))